In [1]:
import requests as re
import os
import pandas as pd
import numpy as np
from datetime import timedelta, date

In [49]:
weather_api = os.environ['weather_api_key']

In [35]:
flights = pd.read_csv('flights_test.csv')

In [36]:
flights['fl_date'] = pd.to_datetime(flights['fl_date'],unit='ms').dt.date

In [37]:
flights_samp = flights.sample(n=20000)

In [38]:
flights_samp = flights_samp.reset_index(drop=True)

In [39]:
flights = flights_samp

In [45]:
arr_date = pd.to_datetime(flights['fl_date'])
for date in range(len(arr_date)):
    if flights['crs_dep_time'][date] > flights['crs_arr_time'][date]:
        arr_date[date] = arr_date[date] + timedelta(days=1)
flights['arr_date'] = arr_date
flights['arr_date'] = flights['arr_date'].astype(str)

In [51]:
flights['fl_date'] = flights['fl_date'].astype(str)

In [53]:
dates_list = flights['fl_date'].unique().tolist()

In [42]:
airport_list = flights['origin'].unique().tolist()

In [57]:
condition_list = []
for date in dates_list:
    for airport in airport_list:
        url = 'http://api.weatherapi.com/v1/history.json?key='+weather_api+'&q='+airport+'&dt='+date+'&hour='+'12'
        r = re.get(url)
        res = r.json()
        temp_list = [date,airport,res['forecast']['forecastday'][0]['hour'][0]['condition']['text']]
        condition_list.append(temp_list)

In [58]:
len(condition_list)

1725

In [60]:
condition_df = pd.DataFrame(data=condition_list,columns=['fl_date','origin', 'dep_condition'])

In [63]:
dest_list = flights['dest'].unique().tolist()

In [47]:
arr_list = flights['arr_date'].unique().tolist()

In [64]:
arr_condition_list = []
for date in arr_list:
    for airport in dest_list:
        url = 'http://api.weatherapi.com/v1/history.json?key='+weather_api+'&q='+airport+'&dt='+date+'&hour='+'12'
        r = re.get(url)
        res = r.json()
        temp_list = [date,airport,res['forecast']['forecastday'][0]['hour'][0]['condition']['text']]
        arr_condition_list.append(temp_list)

In [66]:
flights


,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_date
0,2020-01-04,AA,AA,AA,2670,AA,N730US,2670,10821,BWI,...,11057,CLT,"Charlotte, NC",635,823,N,108,1,361,2020-01-04
1,2020-01-05,DL,DL_CODESHARE,DL,5119,9E,N904XJ,5119,13487,MSP,...,15016,STL,"St. Louis, MO",1259,1442,N,103,1,448,2020-01-05
2,2020-01-03,AA,AA_CODESHARE,AA,5138,OH,N218PS,5138,11278,DCA,...,11995,GSO,"Greensboro/High Point, NC",1515,1626,N,71,1,248,2020-01-03
3,2020-01-04,UA,UA,UA,303,UA,N820UA,303,11278,DCA,...,13930,ORD,"Chicago, IL",845,1002,N,137,1,612,2020-01-04
4,2020-01-02,DL,DL,DL,893,DL,N690DL,893,13204,MCO,...,14869,SLC,"Salt Lake City, UT",700,1009,N,309,1,1931,2020-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2020-01-05,AA,AA_CODESHARE,AA,2952,OO,N742SK,2952,12892,LAX,...,14679,SAN,"San Diego, CA",1339,1435,N,56,1,109,2020-01-05
19996,2020-01-04,DL,DL,DL,1118,DL,N546US,1118,13204,MCO,...,10397,ATL,"Atlanta, GA",1925,2101,N,96,1,404,2020-01-04
19997,2020-01-01,DL,DL,DL,2767,DL,N881DN,2767,11433,DTW,...,14635,RSW,"Fort Myers, FL",1225,1522,N,177,1,1084,2020-01-01
19998,2020-01-04,WN,WN,WN,3945,WN,N924WN,3945,11042,CLE,...,10397,ATL,"Atlanta, GA",725,925,N,120,1,554,2020-01-04


In [67]:
len(arr_condition_list)

2076

In [69]:
arr_condition_df = pd.DataFrame(data=arr_condition_list,columns=['arr_date','dest', 'arr_condition'])

In [71]:
flights_weather = flights.merge(condition_df,how='left',on=['fl_date','origin'] )

In [72]:
flights_weather2 = flights_weather.merge(arr_condition_df,how='left',on=['arr_date','dest'] )

In [75]:
passengers = pd.read_csv('average passengers.csv')

In [79]:
flights_weather2['month'] = pd.DatetimeIndex(flights_weather2['fl_date']).month
flights_weather2['year'] = pd.DatetimeIndex(flights_weather2['fl_date']).year

In [80]:
flights_pass = flights_weather2.merge(passengers,how='left',on=['month','origin','dest'] )

In [81]:
flights_pass['avg_passenger_capacity_usage'] = flights_pass['average_passengers']/flights_pass['average_seats']

In [82]:
flights_pass

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,flights,distance,arr_date,dep_condition,arr_condition,month,year,average_seats,average_passengers,avg_passenger_capacity_usage
0,2020-01-04,AA,AA,AA,2670,AA,N730US,2670,10821,BWI,...,1,361,2020-01-04,Overcast,Partly cloudy,1,2020,6113.571429,4535.857143,0.741932
1,2020-01-05,DL,DL_CODESHARE,DL,5119,9E,N904XJ,5119,13487,MSP,...,1,448,2020-01-05,Partly cloudy,Sunny,1,2020,2169.181818,1433.727273,0.660953
2,2020-01-03,AA,AA_CODESHARE,AA,5138,OH,N218PS,5138,11278,DCA,...,1,248,2020-01-03,Cloudy,Overcast,1,2020,2040.000000,1037.500000,0.508578
3,2020-01-04,UA,UA,UA,303,UA,N820UA,303,11278,DCA,...,1,612,2020-01-04,Overcast,Overcast,1,2020,5733.076923,3353.230769,0.584892
4,2020-01-02,DL,DL,DL,893,DL,N690DL,893,13204,MCO,...,1,1931,2020-01-02,Partly cloudy,Patchy heavy snow,1,2020,3981.200000,3639.400000,0.914146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2020-01-05,AA,AA_CODESHARE,AA,2952,OO,N742SK,2952,12892,LAX,...,1,109,2020-01-05,Cloudy,Sunny,1,2020,3585.700000,2491.400000,0.694816
19996,2020-01-04,DL,DL,DL,1118,DL,N546US,1118,13204,MCO,...,1,404,2020-01-04,Patchy rain possible,Patchy rain possible,1,2020,10788.642857,9196.428571,0.852418
19997,2020-01-01,DL,DL,DL,2767,DL,N881DN,2767,11433,DTW,...,1,1084,2020-01-01,Partly cloudy,Partly cloudy,1,2020,5331.300000,3696.600000,0.693377
19998,2020-01-04,WN,WN,WN,3945,WN,N924WN,3945,11042,CLE,...,1,554,2020-01-04,Overcast,Patchy rain possible,1,2020,6742.166667,5293.500000,0.785133


In [83]:
flights_pass.to_csv('test_set.csv',index=False)